[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/Mix-of-Show-colab/blob/main/Mix_of_Show_hina_colab.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/Mix-of-Show
%cd /content/Mix-of-Show

!pip install -q diffusers==0.20.2 transformers accelerate einops omegaconf
!pip install -q https://download.pytorch.org/whl/cu121/xformers-0.0.22.post7-cp310-cp310-manylinux2014_x86_64.whl

!mkdir -p /content/Mix-of-Show/models/1001_1_EDLoRA_hina_Anyv4_B4_Repeat500
!wget https://huggingface.co/camenduru/Mix-of-Show/resolve/main/single_concept_model/1001_1_EDLoRA_hina_Anyv4_B4_Repeat500/models/edlora_model-latest.pth?download=true -O /content/Mix-of-Show/models/1001_1_EDLoRA_hina_Anyv4_B4_Repeat500/edlora_model-latest.pth

import torch
from diffusers import DPMSolverMultistepScheduler
from mixofshow.pipelines.pipeline_edlora import EDLoRAPipeline, StableDiffusionPipeline
from mixofshow.utils.convert_edlora_to_diffusers import convert_edlora

enable_edlora = True
pipeclass = EDLoRAPipeline if enable_edlora else StableDiffusionPipeline
pretrained_model_path = 'xyn-ai/anything-v4.0'
scheduler = DPMSolverMultistepScheduler.from_pretrained(pretrained_model_path, subfolder='scheduler')
pipe = pipeclass.from_pretrained(pretrained_model_path, scheduler=scheduler, torch_dtype=torch.float16).to('cuda')
edlora_path = '/content/Mix-of-Show/models/1001_1_EDLoRA_hina_Anyv4_B4_Repeat500/edlora_model-latest.pth'
pipe, new_concept_cfg = convert_edlora(pipe, torch.load(edlora_path), enable_edlora=enable_edlora, alpha=1.0)
pipe.set_new_concept_cfg(new_concept_cfg)
pipe.unet.eval()
pipe.text_encoder.eval()

In [ ]:
TOK = '<hina1> <hina2>'  # the TOK is the concept name when training lora/edlora
prompt = f'a {TOK} in front of eiffel tower, 4K, high quality, high resolution'
negative_prompt = 'longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality'
image = pipe(prompt, negative_prompt=negative_prompt, height=512, width=512, num_inference_steps=50, guidance_scale=7.5).images[0]
image.save('res.jpg')
image